#Importing Dependencies and Libraries

In [ ]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime

Token = 'AAKdzwnUOEsfzxNszHjkMNTTkHuxDnLm'


import seaborn as sns 
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
tqdm.pandas()
from haversine import haversine

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Saving the Data
def SaveToCSV(df,path):
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f,index = False)

In [ ]:
def NearestStation(lat,long,df): 
    yo = df
    coords = (lat,long)
    yo["Dist"]= yo.Coordinates.apply(lambda x: haversine(coords,x))
    #yo.reset_index(inplace=True)
    closest = yo.loc[yo["Dist"].idxmin()]
    return(str(closest.ID),float(closest.Dist))

#Input Data

##Weather Stations

In [ ]:
weather_stations = pd.read_csv('/content/drive/MyDrive/298DataFiles/us_ca_stationdata.csv',
                               header=None,names=["ID","LATITUDE","LONGITUDE","ELEVATION","STATE","NAME","GSN FLAG","HCN/CRN FLAG","WMO ID"]) \
                               [['ID',	'LATITUDE',	'LONGITUDE','NAME','ELEVATION']]
weather_stations['Coordinates'] = list(zip(weather_stations['LATITUDE'], weather_stations['LONGITUDE']))
print(weather_stations.shape)
weather_stations.head()

(2923, 6)


,ID,LATITUDE,LONGITUDE,NAME,ELEVATION,Coordinates
0,US1CAAL0001,37.8123,-122.2160,IEDMONT 1.0 SE,113.4,"(37.8123, -122.216)"
1,US1CAAL0002,37.7075,-122.0687,ASTRO VALLEY 0.5 WSW,87.5,"(37.7075, -122.0687)"
2,US1CAAL0003,37.7169,-122.0585,ASTRO VALLEY 0.4 NNE,163.4,"(37.7169, -122.0585)"
3,US1CAAL0004,37.6483,-121.8745,LEASANTON 1.8 SSE,107.0,"(37.6483, -121.8745)"
4,US1CAAL0006,37.8319,-122.2503,AKLAND 4.4 NNW,58.2,"(37.8319, -122.2503)"


## Common Coordinates

In [ ]:
Counties = ['SantaBarbara','SanDiego']
directory = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/LiDAR_Presampled_Data/'

CommonCoordinates = pd.DataFrame()
for County in Counties:
  #Common Coordinates
  CommonCoordinates_file_path = directory+'/'+County+'CommonCoordinates.csv'
  temp_df = pd.read_csv(CommonCoordinates_file_path)
  temp_df['CommonCoordinates'] = list(zip(temp_df['Latitude'], temp_df['Longitude']))
  temp_df['County'] = County
  CommonCoordinates = CommonCoordinates.append(temp_df)

CommonCoordinates

,Latitude,Longitude,CommonCoordinates,County
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara
1,34.4335,-119.9482,"(34.4335, -119.9482)",SantaBarbara
2,34.4336,-119.9482,"(34.4336, -119.9482)",SantaBarbara
3,34.4337,-119.9482,"(34.4337, -119.9482)",SantaBarbara
4,34.4338,-119.9482,"(34.4338, -119.9482)",SantaBarbara
...,...,...,...,...
1790,32.5386,-117.1249,"(32.5386, -117.1249)",SanDiego
1791,32.5404,-117.1249,"(32.5404, -117.1249)",SanDiego
1792,32.5366,-117.1248,"(32.5366, -117.1248)",SanDiego
1793,32.5357,-117.1247,"(32.5357, -117.1247)",SanDiego


# Finding nearest weather Stations

##Flitering Relevant Stations

In [ ]:
rlvnt_stations = weather_stations[weather_stations[["LATITUDE","LONGITUDE"]].round(decimals = 1).apply(tuple, 1).isin( \
                                  CommonCoordinates[["Latitude","Longitude"]].round(decimals = 1).apply(tuple, 1))]
print(rlvnt_stations.shape)
rlvnt_stations.head()

(32, 6)


,ID,LATITUDE,LONGITUDE,NAME,ELEVATION,Coordinates
635,US1CASB0001,34.4397,-119.8961,OLETA 3.7 WNW,48.2,"(34.4397, -119.8961)"
637,US1CASB0004,34.4500,-119.7000,ANTA BARBARA 1.9 NE,247.2,"(34.45, -119.7)"
643,US1CASB0017,34.4355,-119.7735,ANTA BARBARA 3.0 W,24.1,"(34.4355, -119.7735)"
684,US1CASD0003,32.5738,-117.0625,MPERIAL BEACH 3.3 E,42.1,"(32.5738, -117.0625)"
696,US1CASD0015,32.8254,-117.2397,A JOLLA 2.5SE,154.8,"(32.8254, -117.2397)"


##Closest Stations

In [ ]:
CommonCoordinates['ClosestStationID'], CommonCoordinates['Distance'] = \
                            zip(*CommonCoordinates.progress_apply(
                                lambda x: NearestStation(float(x['Latitude']),float(x['Longitude']),rlvnt_stations), axis=1))
CommonCoordinates

100%|██████████| 40536/40536 [00:46<00:00, 872.77it/s]


,Latitude,Longitude,CommonCoordinates,County,ClosestStationID,Distance
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001,4.832374
1,34.4335,-119.9482,"(34.4335, -119.9482)",SantaBarbara,US1CASB0001,4.827489
2,34.4336,-119.9482,"(34.4336, -119.9482)",SantaBarbara,US1CASB0001,4.825911
3,34.4337,-119.9482,"(34.4337, -119.9482)",SantaBarbara,US1CASB0001,4.824357
4,34.4338,-119.9482,"(34.4338, -119.9482)",SantaBarbara,US1CASB0001,4.822829
...,...,...,...,...,...,...
1790,32.5386,-117.1249,"(32.5386, -117.1249)",SanDiego,USW00093115,3.217714
1791,32.5404,-117.1249,"(32.5404, -117.1249)",SanDiego,USW00093115,3.023732
1792,32.5366,-117.1248,"(32.5366, -117.1248)",SanDiego,USW00093115,3.431995
1793,32.5357,-117.1247,"(32.5357, -117.1247)",SanDiego,USW00093115,3.527657


In [ ]:
CommonCoordinates.Distance.max()

15.506203154776603

In [ ]:
CommonCoordinates.Distance.min()

0.2054887515437515

In [ ]:
ClosestStationIDs = list(CommonCoordinates.ClosestStationID.unique())
ClosestStationIDs

['US1CASB0001',
 'USW00053152',
 'USW00023190',
 'US1CASB0017',
 'USC00047902',
 'US1CASD0030',
 'US1CASD0146',
 'USC00042350',
 'US1CASD0115',
 'US1CASD0055',
 'US1CASD0060',
 'USC00047741',
 'USC00041252',
 'USW00093112',
 'USC00041758',
 'USW00093115']

#Extacting Precipitation Data

In [ ]:
#stationids with error : US1CASB0017,US1CASD0146,USC00042350,US1CASD0115,US1CASD0055,US1CASD0060,USC00041252

ClosestStationIDs = ['US1CASB0001',
 'USW00053152',
 'USW00023190',
 'USC00047902',
 'US1CASD0030',
 'USC00047741',
 'USW00093112',
 'USC00041758',
 'USW00093115']

In [ ]:
temperature_df = pd.DataFrame()
precp_df = pd.DataFrame()
station_id_df = pd.DataFrame()

#for each year from 2009 to 2020
for year in range(2009,2020+1):
  for station in ClosestStationIDs:
    year = str(year)
    print('working on year '+year+', '+station)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&limit=1000&stationid=GHCND:'+station+'&startdate='
                      +year+'-01-01&enddate='+year+'-12-31',headers={'token':Token})
    print(r)
    #load the api response as a json
    d = json.loads(r.text)
    
    #initialize lists to store data
    dates_temp = []
    dates_prcp = []
    dates_stationID = []
    temps = []
    prcp = []
    station_id = []
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]
    #get all items in the response which are precipitation readings
    prep_tempry = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all precipitation readings
    dates_prcp += [item['date'] for item in prep_tempry]
    #get the actual precipitation from all precipitation readings
    prcp += [item['value'] for item in prep_tempry]


    temperature_tempdf = pd.DataFrame()
    prcp_tempdf = pd.DataFrame()
    station_id_tempdf = pd.DataFrame()

    temperature_tempdf['date'] = dates_temp
    temperature_tempdf['avgtemp'] = temps
    temperature_tempdf['stationid'] = station
    if len(temperature_tempdf)!=12:
      print('Temp Data Missing')
      print('rows missing: ',12-len(temperature_tempdf))

    prcp_tempdf['date'] = dates_prcp
    prcp_tempdf['prcp'] = prcp
    prcp_tempdf['stationid'] = station
    if len(prcp_tempdf)!=12:
      print('precp Data Missing')
      print('rows missing: ',12-len(prcp_tempdf))



    temperature_df = temperature_df.append(temperature_tempdf)
    precp_df = precp_df.append(prcp_tempdf)

    


working on year 2009, US1CASB0001
<Response [200]>
Temp Data Missing
rows missing:  12
precp Data Missing
rows missing:  8
working on year 2009, USW00053152
<Response [200]>
working on year 2009, USW00023190
<Response [200]>
working on year 2009, USC00047902
<Response [200]>
Temp Data Missing
rows missing:  2
precp Data Missing
rows missing:  2
working on year 2009, US1CASD0030
<Response [200]>
Temp Data Missing
rows missing:  12
precp Data Missing
rows missing:  2
working on year 2009, USC00047741
<Response [200]>
Temp Data Missing
rows missing:  7
working on year 2009, USW00093112
<Response [200]>
Temp Data Missing
rows missing:  1
working on year 2009, USC00041758
<Response [200]>
Temp Data Missing
rows missing:  2
working on year 2009, USW00093115
<Response [200]>
precp Data Missing
rows missing:  1
working on year 2010, US1CASB0001
<Response [200]>
Temp Data Missing
rows missing:  12
working on year 2010, USW00053152
<Response [200]>
working on year 2010, USW00023190
<Response [20

##Creating a DataFrame for storing Precp & Avg Temp

In [ ]:
temp_prep_raw_df = pd.DataFrame({'date':pd.date_range(start='2009-01-01', end='2020-12-01', freq='MS')})
temp_prep_raw_df = (pd.concat([temp_prep_raw_df] * len(ClosestStationIDs), keys=ClosestStationIDs)
   .reset_index(level=1, drop=True)
   .rename_axis('stationid')
   .reset_index()
)
temp_prep_raw_df

,stationid,date
0,US1CASB0001,2009-01-01
1,US1CASB0001,2009-02-01
2,US1CASB0001,2009-03-01
3,US1CASB0001,2009-04-01
4,US1CASB0001,2009-05-01
...,...,...
1291,USW00093115,2020-08-01
1292,USW00093115,2020-09-01
1293,USW00093115,2020-10-01
1294,USW00093115,2020-11-01


In [ ]:
temperature_df['date'] = [datetime.strptime(d,"%Y-%m-%dT%H:%M:%S") for d in temperature_df.date]
temp_prep_raw_df = pd.merge(temp_prep_raw_df,temperature_df,on=['date','stationid'],how='left')
temp_prep_raw_df.avgtemp = temp_prep_raw_df.groupby(['date','stationid'])['avgtemp'].apply(lambda x: x.fillna(x.mean()))
temp_prep_raw_df.avgtemp = temp_prep_raw_df.avgtemp.fillna(temp_prep_raw_df.avgtemp.mean())
temp_prep_raw_df

,stationid,date,avgtemp
0,US1CASB0001,2009-01-01,16.821993
1,US1CASB0001,2009-02-01,16.821993
2,US1CASB0001,2009-03-01,16.821993
3,US1CASB0001,2009-04-01,16.821993
4,US1CASB0001,2009-05-01,16.821993
...,...,...,...
1291,USW00093115,2020-08-01,16.821993
1292,USW00093115,2020-09-01,16.821993
1293,USW00093115,2020-10-01,16.821993
1294,USW00093115,2020-11-01,16.821993


In [ ]:
#precp_df['date'] = [datetime.strptime(d,"%Y-%m-%dT%H:%M:%S") for d in precp_df.date]
temp_prep_raw_df = pd.merge(temp_prep_raw_df,precp_df,on=['date','stationid'],how='left')
temp_prep_raw_df.prcp = temp_prep_raw_df.groupby(['date','stationid'])['prcp'].apply(lambda x: x.fillna(x.mean()))
temp_prep_raw_df.prcp = temp_prep_raw_df.prcp.fillna(temp_prep_raw_df.prcp.mean())
temp_prep_raw_df

,stationid,date,avgtemp,prcp
0,US1CASB0001,2009-01-01,16.821993,24.207605
1,US1CASB0001,2009-02-01,16.821993,24.207605
2,US1CASB0001,2009-03-01,16.821993,24.207605
3,US1CASB0001,2009-04-01,16.821993,24.207605
4,US1CASB0001,2009-05-01,16.821993,24.207605
...,...,...,...,...
1291,USW00093115,2020-08-01,16.821993,0.000000
1292,USW00093115,2020-09-01,16.821993,0.000000
1293,USW00093115,2020-10-01,16.821993,0.000000
1294,USW00093115,2020-11-01,16.821993,0.000000


In [ ]:
# #cast string date to datetime 
# temp_prep_raw_df['date'] = [datetime.strptime(d,"%Y-%m-%dT%H:%M:%S") for d in temp_prep_raw_df.date]
#convert temperature from Celsius to Fahrenheit
temp_prep_raw_df['avgTemp_F'] = [float(v)*1.8 + 32 for v in temp_prep_raw_df.avgtemp]
temp_prep_raw_df['prcp_inch'] = [round(v/25.4,3) for v in temp_prep_raw_df.prcp] #convert millimeters to inch
temp_prep_raw_df = temp_prep_raw_df.rename(columns = {'avgtemp':'avgtemp_C','prcp':'prcp_mm'})
temp_prep_raw_df

,stationid,date,avgtemp_C,prcp_mm,avgTemp_F,prcp_inch
0,US1CASB0001,2009-01-01,16.821993,24.207605,62.279588,0.953
1,US1CASB0001,2009-02-01,16.821993,24.207605,62.279588,0.953
2,US1CASB0001,2009-03-01,16.821993,24.207605,62.279588,0.953
3,US1CASB0001,2009-04-01,16.821993,24.207605,62.279588,0.953
4,US1CASB0001,2009-05-01,16.821993,24.207605,62.279588,0.953
...,...,...,...,...,...,...
1291,USW00093115,2020-08-01,16.821993,0.000000,62.279588,0.000
1292,USW00093115,2020-09-01,16.821993,0.000000,62.279588,0.000
1293,USW00093115,2020-10-01,16.821993,0.000000,62.279588,0.000
1294,USW00093115,2020-11-01,16.821993,0.000000,62.279588,0.000


In [ ]:
SaveToCSV(temp_prep_raw_df,path = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/Temp_Prcp_TwoCounties_AllYears.csv')

#Preparing for merge with other data

In [ ]:
Temp_Prcp_TwoCounties_AllYears = pd.read_csv('/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/Temp_Prcp_TwoCounties_AllYears.csv')
Temp_Prcp_TwoCounties_AllYears

,stationid,date,avgtemp_C,prcp_mm,avgTemp_F,prcp_inch
0,US1CASB0001,2009-01-01,16.821993,24.207605,62.279588,0.953
1,US1CASB0001,2009-02-01,16.821993,24.207605,62.279588,0.953
2,US1CASB0001,2009-03-01,16.821993,24.207605,62.279588,0.953
3,US1CASB0001,2009-04-01,16.821993,24.207605,62.279588,0.953
4,US1CASB0001,2009-05-01,16.821993,24.207605,62.279588,0.953
...,...,...,...,...,...,...
1291,USW00093115,2020-08-01,16.821993,0.000000,62.279588,0.000
1292,USW00093115,2020-09-01,16.821993,0.000000,62.279588,0.000
1293,USW00093115,2020-10-01,16.821993,0.000000,62.279588,0.000
1294,USW00093115,2020-11-01,16.821993,0.000000,62.279588,0.000


In [ ]:
Temp_Prcp_TwoCounties_AllYears.stationid.unique()

array(['US1CASB0001', 'USW00053152', 'USW00023190', 'USC00047902',
       'US1CASD0030', 'USC00047741', 'USW00093112', 'USC00041758',
       'USW00093115'], dtype=object)

##Flitering available Stations

In [ ]:
#stationids with error : US1CASB0017,US1CASD0146,USC00042350,US1CASD0115,US1CASD0055,US1CASD0060,USC00041252

ClosestStationIDs = ['US1CASB0001',
 'USW00053152',
 'USW00023190',
 'USC00047902',
 'US1CASD0030',
 'USC00047741',
 'USW00093112',
 'USC00041758',
 'USW00093115']

available_stations = weather_stations[weather_stations['ID'].isin(ClosestStationIDs)]
print(available_stations.shape)
available_stations.head()

(9, 6)


,ID,LATITUDE,LONGITUDE,NAME,ELEVATION,Coordinates
635,US1CASB0001,34.4397,-119.8961,OLETA 3.7 WNW,48.2,"(34.4397, -119.8961)"
709,US1CASD0030,32.8257,-117.2501,A JOLLA 2.1 SE,149.7,"(32.8257, -117.2501)"
1440,USC00041758,32.6400,-117.0858,HULA VISTA,17.1,"(32.64, -117.0858)"
2143,USC00047741,32.7653,-117.2314,AN DIEGO SEAWORLD,4.3,"(32.7653, -117.2314)"
2176,USC00047902,34.4167,-119.6844,ANTA BARBARA,4.9,"(34.4167, -119.6844)"


##Update the CommonCoordinates table with closest available stations

In [ ]:
CommonCoordinates['ClosestStationID'], CommonCoordinates['Distance'] = \
                            zip(*CommonCoordinates.progress_apply(
                                lambda x: NearestStation(float(x['Latitude']),float(x['Longitude']),available_stations), axis=1))
CommonCoordinates = CommonCoordinates.rename(columns={'ClosestStationID':'stationid'})
CommonCoordinates = CommonCoordinates.drop('Distance',axis=1)
CommonCoordinates

100%|██████████| 40536/40536 [00:35<00:00, 1147.36it/s]


,Latitude,Longitude,CommonCoordinates,County,stationid
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001
1,34.4335,-119.9482,"(34.4335, -119.9482)",SantaBarbara,US1CASB0001
2,34.4336,-119.9482,"(34.4336, -119.9482)",SantaBarbara,US1CASB0001
3,34.4337,-119.9482,"(34.4337, -119.9482)",SantaBarbara,US1CASB0001
4,34.4338,-119.9482,"(34.4338, -119.9482)",SantaBarbara,US1CASB0001
...,...,...,...,...,...
1790,32.5386,-117.1249,"(32.5386, -117.1249)",SanDiego,USW00093115
1791,32.5404,-117.1249,"(32.5404, -117.1249)",SanDiego,USW00093115
1792,32.5366,-117.1248,"(32.5366, -117.1248)",SanDiego,USW00093115
1793,32.5357,-117.1247,"(32.5357, -117.1247)",SanDiego,USW00093115


In [ ]:
SaveToCSV(CommonCoordinates,path = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/CommonCoordinatesWithStations.csv')

In [ ]:
#getting coordinates from actual data for merging
Prcp_Temp_Df = pd.merge(CommonCoordinates,Temp_Prcp_TwoCounties_AllYears,on=['stationid'],how='left')
Prcp_Temp_Df

,Latitude,Longitude,CommonCoordinates,County,stationid,date,avgtemp_C,prcp_mm,avgTemp_F,prcp_inch
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001,2009-01-01,16.821993,24.207605,62.279588,0.953
1,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001,2009-02-01,16.821993,24.207605,62.279588,0.953
2,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001,2009-03-01,16.821993,24.207605,62.279588,0.953
3,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001,2009-04-01,16.821993,24.207605,62.279588,0.953
4,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,US1CASB0001,2009-05-01,16.821993,24.207605,62.279588,0.953
...,...,...,...,...,...,...,...,...,...,...
5837179,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,USW00093115,2020-08-01,16.821993,0.000000,62.279588,0.000
5837180,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,USW00093115,2020-09-01,16.821993,0.000000,62.279588,0.000
5837181,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,USW00093115,2020-10-01,16.821993,0.000000,62.279588,0.000
5837182,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,USW00093115,2020-11-01,16.821993,0.000000,62.279588,0.000


In [ ]:
SaveToCSV(Prcp_Temp_Df,path = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/Temp_Prcp_SB_SD_AllCoordinates_09_20.csv')